In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
import random
import numpy as np

# 1. Cihaz Ayarı (Varsa GPU kullanalım)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Kullanılan Cihaz: {device}")

# 2. MNIST Verisini İndirme ve Hazırlama
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
train_data = datasets.MNIST('./data', train=True, download=True, transform=transform)

# 3. Federe Simülasyonu için Veriyi Bölme (Sanal İstemciler)
# Veriyi Client A ve Client B arasında rastgele bölüyoruz (Her birine 2000'er örnek verelim, hızlı olsun)
indices = list(range(len(train_data)))
random.shuffle(indices)

client_a_indices = indices[:2000]
client_b_indices = indices[2000:4000]

client_a_loader = DataLoader(Subset(train_data, client_a_indices), batch_size=32, shuffle=True)
client_b_loader = DataLoader(Subset(train_data, client_b_indices), batch_size=32, shuffle=True)

print(f"Veri Hazır: Client A ({len(client_a_indices)} örnek), Client B ({len(client_b_indices)} örnek)")
#veri setlerinin standart sapmalarına dikkat et

Kullanılan Cihaz: cpu


100%|██████████| 9.91M/9.91M [00:00<00:00, 57.9MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 1.67MB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 14.6MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 6.94MB/s]

Veri Hazır: Client A (2000 örnek), Client B (2000 örnek)


In [ ]:
class DynamicNet(nn.Module):
    def __init__(self, hidden_size):
        super(DynamicNet, self).__init__()
        # Mimarinin "Geni": hidden_size
        self.hidden_size = hidden_size

        self.fc1 = nn.Linear(28 * 28, hidden_size) # Giriş Katmanı
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, 10)      # Çıkış Katmanı (0-9 arası rakamlar)

    def forward(self, x):
        x = x.view(-1, 28 * 28) # Düzleştirme
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Modeli eğitip başarısını (Fitness) döndüren yardımcı fonksiyon (İstemci tarafı simülasyonu)
def train_and_evaluate(model, dataloader):
    model.to(device)
    optimizer = optim.SGD(model.parameters(), lr=0.01)
    criterion = nn.CrossEntropyLoss()

    # 1 Epoch Eğitim (Hızlı olsun diye az tutuyoruz)
    model.train()
    for data, target in dataloader:
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

    # Değerlendirme (Accuracy Hesaplama)
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data, target in dataloader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            _, predicted = torch.max(output.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()

    accuracy = 100 * correct / total
    return accuracy

In [ ]:
# --- BAŞLANGIÇ AYARLARI ---
POPULATION_SIZE = 5
GENERATIONS = 3 # Kaç nesil evrimleşsin?

# 1. İlk Popülasyonu Rastgele Yarat (Genler: Nöron sayıları)
# Örn: Biri 10 nöronlu, biri 100 nöronlu...
population_genes = [random.randint(10, 100) for _ in range(POPULATION_SIZE)]

print(f"--- BAŞLANGIÇ POPÜLASYONU (GENLER/NÖRON SAYILARI) ---")
print(population_genes)
print("-" * 50)

for generation in range(GENERATIONS):
    print(f"\n>>> NESİL {generation + 1} BAŞLIYOR...")
    fitness_scores = []

    # Her bir "Birey" (Model) için döngü
    for i, gene in enumerate(population_genes):
        # A. Sunucu modeli gen'e göre inşa eder
        model = DynamicNet(hidden_size=gene)

        # B. Modeli İstemcilere gönderir ve eğitir (Burada ortalama başarıyı alıyoruz)
        # Client A başarısı
        acc_a = train_and_evaluate(model, client_a_loader)
        # Client B başarısı (Sanki farklı bir model kopyasıymış gibi sıfırdan eğitiyoruz simülasyon gereği)
        # Not: Gerçekte ağırlıklar paylaşılır, burada basit tutuyoruz.
        model_b = DynamicNet(hidden_size=gene)
        acc_b = train_and_evaluate(model_b, client_b_loader)

        # Ortalama Başarım (Fitness Skoru)
        avg_fitness = (acc_a + acc_b) / 2
        fitness_scores.append((gene, avg_fitness))

        print(f"Model {i+1} (Gen: {gene} nöron) -> Başarım: %{avg_fitness:.2f}")

    # C. Doğal Seçilim (En iyi 2 modeli seç)
    fitness_scores.sort(key=lambda x: x[1], reverse=True) # Puana göre sırala
    best_parents = fitness_scores[:2] # En iyi 2 tanesi "Ebeveyn" olur

    print(f"  -> Kazananlar: {best_parents[0][0]} nöron ve {best_parents[1][0]} nöron")

    # D. Yeni Nesil Üretimi (Crossover ve Mutasyon)
    new_population = []

    # 1. Elitizm: En iyileri aynen koru
    new_population.append(best_parents[0][0])
    new_population.append(best_parents[1][0])

    # 2. Çocuk Üretme (Geriye kalan 3 kişi için)
    while len(new_population) < POPULATION_SIZE:
        # Crossover: İki ebeveynin ortalamasını al
        parent1_gene = best_parents[0][0]
        parent2_gene = best_parents[1][0]
        child_gene = int((parent1_gene + parent2_gene) / 2)

        # Mutasyon: Rastgele +/- değişim yap (%20 oranında)
        mutation = random.randint(-10, 10)
        child_gene += mutation

        # Negatif olmasın diye kontrol
        child_gene = max(5, child_gene)

        new_population.append(child_gene)

    # Popülasyonu güncelle
    population_genes = new_population
    print(f"  -> Yeni Nesil Genleri: {population_genes}")

print("\n--- EVRİM TAMAMLANDI ---")
print(f"En son ve en güçlü mimari: {population_genes[0]} nöronlu model.")

--- BAŞLANGIÇ POPÜLASYONU (GENLER/NÖRON SAYILARI) ---
[31, 32, 70, 61, 78]
--------------------------------------------------

>>> NESİL 1 BAŞLIYOR...
Model 1 (Gen: 31 nöron) -> Başarım: %69.10
Model 2 (Gen: 32 nöron) -> Başarım: %70.68
Model 3 (Gen: 70 nöron) -> Başarım: %73.78
Model 4 (Gen: 61 nöron) -> Başarım: %75.07
Model 5 (Gen: 78 nöron) -> Başarım: %75.40
  -> Kazananlar: 78 nöron ve 61 nöron
  -> Yeni Nesil Genleri: [78, 61, 64, 65, 69]

>>> NESİL 2 BAŞLIYOR...
Model 1 (Gen: 78 nöron) -> Başarım: %74.10
Model 2 (Gen: 61 nöron) -> Başarım: %73.38
Model 3 (Gen: 64 nöron) -> Başarım: %71.42
Model 4 (Gen: 65 nöron) -> Başarım: %68.68
Model 5 (Gen: 69 nöron) -> Başarım: %72.43
  -> Kazananlar: 78 nöron ve 61 nöron
  -> Yeni Nesil Genleri: [78, 61, 72, 70, 71]

>>> NESİL 3 BAŞLIYOR...
Model 1 (Gen: 78 nöron) -> Başarım: %73.85
Model 2 (Gen: 61 nöron) -> Başarım: %72.40
Model 3 (Gen: 72 nöron) -> Başarım: %74.83
Model 4 (Gen: 70 nöron) -> Başarım: %73.47
Model 5 (Gen: 71 nöron) -> Ba

In [ ]:
# --- ADIM 4: HAFIZALI EVRİM (WEIGHT TRANSFER) ---

# Yardımcı Fonksiyon: Farklı boyuttaki iki model arasında bilgi aktarımı
def transfer_knowledge(parent_model, child_model):
    # 1. İlk Katman Transferi (Input -> Hidden)
    # Parent ve Child'ın nöron sayılarına bakıyoruz
    p_neurons = parent_model.fc1.weight.data.shape[0]
    c_neurons = child_model.fc1.weight.data.shape[0]

    # Hangisi küçükse o kadarını kopyalayabiliriz
    limit = min(p_neurons, c_neurons)

    # Ağırlıkları kopyala (Sadece sığan kısmı)
    child_model.fc1.weight.data[:limit, :] = parent_model.fc1.weight.data[:limit, :]
    child_model.fc1.bias.data[:limit] = parent_model.fc1.bias.data[:limit]

    # 2. İkinci Katman Transferi (Hidden -> Output)
    # Burada hidden katman "giriş" olduğu için sütunları kopyalıyoruz
    child_model.fc2.weight.data[:, :limit] = parent_model.fc2.weight.data[:, :limit]
    # (Not: Çıkış katmanı bias'ı her zaman 10 olduğu için direkt kopyalanır)
    child_model.fc2.bias.data = parent_model.fc2.bias.data

    return child_model

# --- GELİŞMİŞ EVRİM DÖNGÜSÜ ---
print("--- BİLGİ AKTARIMLI EVRİM BAŞLIYOR ---")

# En iyi modelleri saklamak için sözlük (Gen -> Modelin Kendisi)
best_models_cache = {}

# Başlangıç popülasyonu (yine rastgele)
current_genes = [random.randint(20, 100) for _ in range(5)]

for generation in range(3): # 3 Nesil
    print(f"\n>>> NESİL {generation + 1} (Bilgi Aktarımı Aktif)")

    results = []

    for gene in current_genes:
        # A. Modeli Yarat
        model = DynamicNet(hidden_size=gene)

        # B. Eğer Ebeveyni Varsa Bilgiyi Aktar!
        # (Önceki nesilden en iyi modele yakın bir model mi?)
        if len(best_models_cache) > 0:
            # Cache'deki en iyi modeli bul (Parent)
            best_parent_gene = list(best_models_cache.keys())[0]
            parent_model = best_models_cache[best_parent_gene]

            # Bilgi Nakli Yap (Ameliyat)
            model = transfer_knowledge(parent_model, model)
            # Not: Burada ekrana basmıyoruz ama şu an model "doğuştan bilgili"

        # C. Eğitime Kaldığı Yerden Devam Et (Fine-tuning)
        # Normalde %10 ile başlardı, şimdi belki %60 ile başlayacak
        acc = train_and_evaluate(model, client_a_loader) # Sadece Client A ile test edelim hız için

        results.append((gene, acc, model))
        print(f"Gen: {gene} nöron -> Başarım: %{acc:.2f}")

    # D. Sıralama ve Seçme
    results.sort(key=lambda x: x[1], reverse=True)
    winner_gene, winner_acc, winner_model = results[0]

    print(f"  -> LİDER: {winner_gene} nöron (%{winner_acc:.2f})")

    # E. Lideri Hafızaya Al (Bir sonraki nesil buna benzeyecek)
    best_models_cache = {winner_gene: winner_model}

    # F. Yeni Nesil Üretimi (Mutasyon)
    new_genes = []
    new_genes.append(winner_gene) # Lideri koru

    while len(new_genes) < 5:
        # Liderin etrafında mutasyonlar yap
        mutation = random.randint(-5, 10)
        child = winner_gene + mutation
        child = max(10, child) # 10'dan küçük olmasın
        new_genes.append(child)

    current_genes = new_genes

--- BİLGİ AKTARIMLI EVRİM BAŞLIYOR ---

>>> NESİL 1 (Bilgi Aktarımı Aktif)
Gen: 54 nöron -> Başarım: %73.45
Gen: 93 nöron -> Başarım: %73.00
Gen: 66 nöron -> Başarım: %69.65
Gen: 41 nöron -> Başarım: %69.65
Gen: 55 nöron -> Başarım: %65.25
  -> LİDER: 54 nöron (%73.45)

>>> NESİL 2 (Bilgi Aktarımı Aktif)
Gen: 54 nöron -> Başarım: %82.60
Gen: 51 nöron -> Başarım: %81.85
Gen: 61 nöron -> Başarım: %83.45
Gen: 51 nöron -> Başarım: %84.70
Gen: 60 nöron -> Başarım: %84.15
  -> LİDER: 51 nöron (%84.70)

>>> NESİL 3 (Bilgi Aktarımı Aktif)
Gen: 51 nöron -> Başarım: %85.90
Gen: 53 nöron -> Başarım: %86.65
Gen: 56 nöron -> Başarım: %86.20
Gen: 55 nöron -> Başarım: %86.10
Gen: 60 nöron -> Başarım: %86.70
  -> LİDER: 60 nöron (%86.70)


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
import random
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Kullanılan Cihaz: {device}")

# 1. CIFAR-10 için Veri Ön İşleme (Renkli resimler için istatistikler farklıdır)
stats = ((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)) # RGB Ortalamaları
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(*stats)
])

# 2. CIFAR-10 İndiriliyor
print("CIFAR-10 Verisi indiriliyor (Biraz zaman alabilir)...")
train_data = datasets.CIFAR10('./data_cifar', train=True, download=True, transform=transform)

# 3. Sanal İstemcilere Dağıtım (Daha az veri alalım ki Colab çökmesin)
indices = list(range(len(train_data)))
random.shuffle(indices)

# Her istemciye 1000 resim verelim (İşlemciyi yormamak için az tutuyoruz)
client_a_indices = indices[:1000]
client_b_indices = indices[1000:2000]

client_a_loader = DataLoader(Subset(train_data, client_a_indices), batch_size=32, shuffle=True)
client_b_loader = DataLoader(Subset(train_data, client_b_indices), batch_size=32, shuffle=True)

print(f"Veri Hazır: Client A ve Client B (Uçaklar, Kediler, Arabalar...)")

Kullanılan Cihaz: cpu
CIFAR-10 Verisi indiriliyor (Biraz zaman alabilir)...


100%|██████████| 170M/170M [00:02<00:00, 66.5MB/s]


Veri Hazır: Client A ve Client B (Uçaklar, Kediler, Arabalar...)


In [ ]:
class DynamicCNN(nn.Module):
    def __init__(self, n_filters):
        super(DynamicCNN, self).__init__()
        self.n_filters = n_filters

        # 1. Konvolüsyon Bloğu (Gözler)
        # Giriş 3 Kanal (RGB) -> Çıkış "n_filters" kadar özellik haritası
        self.conv1 = nn.Conv2d(3, n_filters, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(2, 2) # Resmi yarıya küçültür (32x32 -> 16x16)

        # 2. İkinci Blok (Daha derin özellikler)
        # Giriş "n_filters" -> Çıkış "n_filters * 2"
        self.conv2 = nn.Conv2d(n_filters, n_filters*2, kernel_size=3, padding=1)
        # Bir pool daha (16x16 -> 8x8)

        # 3. Karar Bloğu (Beyin - Fully Connected)
        # Resim 8x8 boyutuna düştü. Derinlik ise (n_filters * 2) oldu.
        flatten_size = 8 * 8 * (n_filters * 2)
        self.fc = nn.Linear(flatten_size, 10) # 10 Sınıf (Uçak, Araba...)

    def forward(self, x):
        # Blok 1
        x = self.pool(self.relu(self.conv1(x)))
        # Blok 2
        x = self.pool(self.relu(self.conv2(x)))
        # Düzleştirme
        x = x.view(x.size(0), -1)
        # Karar
        x = self.fc(x)
        return x

# Eğitim Fonksiyonu (Değişmedi)
def train_and_evaluate(model, dataloader):
    model.to(device)
    optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9) # Momentum ekledik, daha hızlı öğrensin
    criterion = nn.CrossEntropyLoss()

    model.train()
    for data, target in dataloader:
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data, target in dataloader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            _, predicted = torch.max(output.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()
    return 100 * correct / total

In [ ]:
# --- EVRİM BAŞLIYOR (CIFAR-10 VERSİYONU) ---
POPULATION_SIZE = 4 # Daha az model (Hız için)
GENERATIONS = 3

# Filtre sayılarını gen olarak atıyoruz (Örn: 16 filtre, 32 filtre...)
# CNN filtreleri genelde 2'nin katları olur, o yüzden listeden seçtirelim
possible_genes = [8, 16, 32, 64]
population_genes = [random.choice(possible_genes) for _ in range(POPULATION_SIZE)]

print(f"--- BAŞLANGIÇ POPÜLASYONU (FİLTRE SAYILARI) ---")
print(population_genes)

for generation in range(GENERATIONS):
    print(f"\n>>> NESİL {generation + 1} BAŞLIYOR...")
    fitness_scores = []

    for i, gene in enumerate(population_genes):
        # A. CNN Modelini Yarat
        model = DynamicCNN(n_filters=gene)

        # B. Eğit ve Test Et (Sadece Client A ile test edelim, hız kazanalım)
        acc = train_and_evaluate(model, client_a_loader)

        fitness_scores.append((gene, acc))
        print(f"Model {i+1} (Filtre: {gene}) -> Başarım: %{acc:.2f}")

    # C. Seçilim
    fitness_scores.sort(key=lambda x: x[1], reverse=True)
    best_gene = fitness_scores[0][0]
    best_acc = fitness_scores[0][1]

    print(f"  -> LİDER: {best_gene} Filtreli Mimari (Acc: %{best_acc:.2f})")

    # D. Yeni Nesil (Basit Mutasyon)
    # Lideri koruyoruz, diğerlerini liderin etrafında türetiyoruz
    new_population = [best_gene]

    while len(new_population) < POPULATION_SIZE:
        # Ya liderin aynısı olsun ya da bir boy büyüğü/küçüğü
        mutation_choice = random.choice([0.5, 1, 2]) # Yarıya düşür, Aynı kal, İkiye katla
        child_gene = int(best_gene * mutation_choice)

        # Sınırları koru (En az 4, en çok 128 filtre)
        child_gene = max(4, min(128, child_gene))
        new_population.append(child_gene)

    population_genes = new_population
    print(f"  -> Yeni Nesil Gen Havuzu: {population_genes}")

print("\n--- TEST BİTTİ ---")
print(f"CIFAR-10 için bulunan en ideal yapı: {population_genes[0]} filtreli CNN.")

--- BAŞLANGIÇ POPÜLASYONU (FİLTRE SAYILARI) ---
[16, 8, 64, 64]

>>> NESİL 1 BAŞLIYOR...
Model 1 (Filtre: 16) -> Başarım: %31.60
Model 2 (Filtre: 8) -> Başarım: %27.00
Model 3 (Filtre: 64) -> Başarım: %32.10
Model 4 (Filtre: 64) -> Başarım: %31.80
  -> LİDER: 64 Filtreli Mimari (Acc: %32.10)
  -> Yeni Nesil Gen Havuzu: [64, 32, 64, 128]

>>> NESİL 2 BAŞLIYOR...
Model 1 (Filtre: 64) -> Başarım: %38.90
Model 2 (Filtre: 32) -> Başarım: %32.60
Model 3 (Filtre: 64) -> Başarım: %41.40
Model 4 (Filtre: 128) -> Başarım: %34.50
  -> LİDER: 64 Filtreli Mimari (Acc: %41.40)
  -> Yeni Nesil Gen Havuzu: [64, 64, 32, 32]

>>> NESİL 3 BAŞLIYOR...
Model 1 (Filtre: 64) -> Başarım: %37.90
Model 2 (Filtre: 64) -> Başarım: %35.00
Model 3 (Filtre: 32) -> Başarım: %36.30
Model 4 (Filtre: 32) -> Başarım: %25.30
  -> LİDER: 64 Filtreli Mimari (Acc: %37.90)
  -> Yeni Nesil Gen Havuzu: [64, 32, 64, 64]

--- TEST BİTTİ ---
CIFAR-10 için bulunan en ideal yapı: 64 filtreli CNN.


In [ ]:
# --- GELİŞMİŞ MODEL: HEM FİLTRE HEM DERİNLİK EVRİMİ ---

class AdvancedDynamicCNN(nn.Module):
    def __init__(self, n_filters, n_layers):
        super(AdvancedDynamicCNN, self).__init__()
        self.n_filters = n_filters
        self.n_layers = n_layers

        # Katmanları tutacak bir liste (ModuleList)
        self.layers = nn.ModuleList()

        # 1. Giriş Katmanı (RGB -> n_filters)
        self.layers.append(nn.Conv2d(3, n_filters, kernel_size=3, padding=1))
        self.layers.append(nn.ReLU())

        # 2. Ara Katmanlar (n_layers kadar ekle)
        # Not: Her katmandan sonra Pooling yaparsak resim çok küçülür (1x1 olur).
        # O yüzden sadece belirli katmanlarda Pooling yapacağız.
        for i in range(n_layers):
            # Filtre sayısını her katmanda aynı tutuyoruz basitlik için
            self.layers.append(nn.Conv2d(n_filters, n_filters, kernel_size=3, padding=1))
            self.layers.append(nn.ReLU())

            # Her 2 katmanda bir resmi küçült (Pooling)
            if i % 2 == 0:
                self.layers.append(nn.MaxPool2d(2, 2))

        # 3. Çıkış Boyutunu Hesaplama (Otomatik)
        # Giriş 32x32. Kaç kere Pool yaptık?
        num_pools = (n_layers // 2) + 1 # +1 de baştaki işlemden gelebilir mantığıyla kabaca hesap
        # Basitlik adına, dinamik olarak düzleştirme yapacağız (Global Average Pooling)
        # Bu yöntem modern CNN'lerde çok kullanılır, resim boyutu ne olursa olsun tek vektör verir.
        self.global_pool = nn.AdaptiveAvgPool2d((1, 1))

        self.fc = nn.Linear(n_filters, 10) # Son karar

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)

        # Global Pooling: (Batch, Filter, 8, 8) -> (Batch, Filter, 1, 1)
        x = self.global_pool(x)
        x = x.view(x.size(0), -1) # Düzleştir
        x = self.fc(x)
        return x

# --- YENİ EVRİM DÖNGÜSÜ ---
print("--- ÇOK BOYUTLU EVRİM (FİLTRE + KATMAN) ---")

# Genom artık bir liste: [Filtre Sayısı, Katman Sayısı]
# Örn: [32, 2] -> 32 Filtreli, 2 Katmanlı
# Örn: [64, 5] -> 64 Filtreli, 5 Katmanlı (Derin)
population = []
for _ in range(4):
    f = random.choice([16, 32, 64])
    l = random.choice([2, 3, 4, 5])
    population.append([f, l])

print(f"Başlangıç Genleri: {population}")

for generation in range(3): # 3 Nesil
    print(f"\n>>> NESİL {generation + 1}")
    scores = []

    for gene in population:
        f, l = gene
        model = AdvancedDynamicCNN(n_filters=f, n_layers=l)

        # Hız için yine Client A ile eğitelim
        # (Önceki train fonksiyonunu kullanıyoruz)
        acc = train_and_evaluate(model, client_a_loader)

        scores.append((gene, acc))
        print(f"Gen: {gene} (Filtre: {f}, Katman: {l}) -> Başarım: %{acc:.2f}")

    # Seçilim
    scores.sort(key=lambda x: x[1], reverse=True)
    best_gene = scores[0][0]
    print(f"  -> LİDER: {best_gene} (Acc: %{scores[0][1]:.2f})")

    # Yeni Nesil (Mutasyon)
    new_pop = [best_gene]
    while len(new_pop) < 4:
        # Ya filtreyi değiştir, ya katmanı
        mutation_type = random.choice(['filter', 'layer'])

        new_f, new_l = best_gene

        if mutation_type == 'filter':
            new_f = random.choice([16, 32, 64, 128])
        else:
            new_l = new_l + random.choice([-1, 1]) # Katman ekle veya çıkar
            new_l = max(2, min(6, new_l)) # 2 ile 6 katman arası sınırla

        new_pop.append([new_f, new_l])

    population = new_pop

--- ÇOK BOYUTLU EVRİM (FİLTRE + KATMAN) ---
Başlangıç Genleri: [[32, 3], [64, 2], [16, 5], [32, 2]]

>>> NESİL 1
Gen: [32, 3] (Filtre: 32, Katman: 3) -> Başarım: %11.50
Gen: [64, 2] (Filtre: 64, Katman: 2) -> Başarım: %19.20
Gen: [16, 5] (Filtre: 16, Katman: 5) -> Başarım: %10.00
Gen: [32, 2] (Filtre: 32, Katman: 2) -> Başarım: %14.70
  -> LİDER: [64, 2] (Acc: %19.20)

>>> NESİL 2
Gen: [64, 2] (Filtre: 64, Katman: 2) -> Başarım: %14.00
Gen: [64, 3] (Filtre: 64, Katman: 3) -> Başarım: %11.50
Gen: [16, 2] (Filtre: 16, Katman: 2) -> Başarım: %9.60
Gen: [64, 2] (Filtre: 64, Katman: 2) -> Başarım: %12.30
  -> LİDER: [64, 2] (Acc: %14.00)

>>> NESİL 3
Gen: [64, 2] (Filtre: 64, Katman: 2) -> Başarım: %11.50
Gen: [64, 2] (Filtre: 64, Katman: 2) -> Başarım: %14.90
Gen: [64, 2] (Filtre: 64, Katman: 2) -> Başarım: %12.00
Gen: [32, 2] (Filtre: 32, Katman: 2) -> Başarım: %11.50
  -> LİDER: [64, 2] (Acc: %14.90)


In [ ]:
# --- DÜZELTİLMİŞ MODEL: BATCH NORM İLE DERİN ÖĞRENME ---

class ProDynamicCNN(nn.Module):
    def __init__(self, n_filters, n_layers):
        super(ProDynamicCNN, self).__init__()
        self.n_filters = n_filters
        self.n_layers = n_layers

        self.layers = nn.ModuleList()

        # 1. Giriş Katmanı
        self.layers.append(nn.Conv2d(3, n_filters, kernel_size=3, padding=1))
        self.layers.append(nn.BatchNorm2d(n_filters)) # <--- SİHİRLİ DOKUNUŞ
        self.layers.append(nn.ReLU())

        # 2. Ara Katmanlar
        for i in range(n_layers):
            self.layers.append(nn.Conv2d(n_filters, n_filters, kernel_size=3, padding=1))
            self.layers.append(nn.BatchNorm2d(n_filters)) # <--- BURASI KRİTİK
            self.layers.append(nn.ReLU())

            # Her 2 katmanda bir pool
            if i % 2 == 0:
                self.layers.append(nn.MaxPool2d(2, 2))

        self.global_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(n_filters, 10)

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        x = self.global_pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

# --- EVRİM DÖNGÜSÜ (Aynı mantık, sadece sınıf adı değişti) ---
print("--- PRO EVRİM (BATCH NORM DESTEKLİ) ---")

population = []
for _ in range(4):
    f = random.choice([32, 64])
    l = random.choice([2, 3, 4])
    population.append([f, l])

# Eğitim fonksiyonunda öğrenme hızını biraz artıralım çünkü Batch Norm hızlı öğrenmeyi sever
def train_fast(model, loader):
    model.to(device)
    # Learning rate'i biraz artırdık (0.005 -> 0.01)
    optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
    criterion = nn.CrossEntropyLoss()
    model.train()
    for data, target in loader:
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

    # Test
    model.eval()
    correct = 0; total = 0
    with torch.no_grad():
        for data, target in loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            _, predicted = torch.max(output.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()
    return 100 * correct / total

# Döngü
for generation in range(3):
    print(f"\n>>> NESİL {generation + 1}")
    scores = []

    for gene in population:
        f, l = gene
        model = ProDynamicCNN(n_filters=f, n_layers=l)
        acc = train_fast(model, client_a_loader) # Hızlandırılmış eğitim
        scores.append((gene, acc))
        print(f"Gen: {gene} -> Başarım: %{acc:.2f}")

    scores.sort(key=lambda x: x[1], reverse=True)
    best_gene = scores[0][0]
    print(f"  -> LİDER: {best_gene} (Acc: %{scores[0][1]:.2f})")

    # Mutasyon
    new_pop = [best_gene]
    while len(new_pop) < 4:
        mutation_type = random.choice(['filter', 'layer'])
        new_f, new_l = best_gene
        if mutation_type == 'filter': new_f = random.choice([32, 64, 128])
        else: new_l = max(2, min(5, new_l + random.choice([-1, 1])))
        new_pop.append([new_f, new_l])
    population = new_pop

--- PRO EVRİM (BATCH NORM DESTEKLİ) ---

>>> NESİL 1
Gen: [64, 3] -> Başarım: %28.40
Gen: [64, 2] -> Başarım: %26.30
Gen: [64, 3] -> Başarım: %28.00
Gen: [64, 2] -> Başarım: %30.50
  -> LİDER: [64, 2] (Acc: %30.50)

>>> NESİL 2
Gen: [64, 2] -> Başarım: %28.00
Gen: [64, 3] -> Başarım: %31.10
Gen: [64, 3] -> Başarım: %29.30
Gen: [128, 2] -> Başarım: %28.40
  -> LİDER: [64, 3] (Acc: %31.10)

>>> NESİL 3
Gen: [64, 3] -> Başarım: %23.90
Gen: [64, 2] -> Başarım: %25.70
Gen: [32, 3] -> Başarım: %29.10
Gen: [64, 2] -> Başarım: %28.40
  -> LİDER: [32, 3] (Acc: %29.10)


In [ ]:
import time

# --- MARATON MODU: GERÇEK PERFORMANS TESTİ ---

# Ayarlar (Süreyi buradan kontrol ediyoruz)
EPOCHS_PER_MODEL = 5  # Her model 5 tur dönecek (Eskiden 1'di)
GENERATIONS = 5       # 5 Nesil boyunca evrimleşecek (Eskiden 3'tü)
POPULATION_SIZE = 4   # Her nesilde 4 farklı model yarışacak

print(f"--- MARATON BAŞLIYOR ---")
print(f"Her model {EPOCHS_PER_MODEL} Epoch eğitilecek.")
print(f"Toplam {GENERATIONS} Nesil sürecek.")
print("Bu işlem biraz zaman alacak, lütfen bekleyin...\n")

# İstatistikleri tutmak için
history = []

# Başlangıç Popülasyonu
population = []
for _ in range(POPULATION_SIZE):
    f = random.choice([32, 64, 128]) # 128'i de oyuna soktuk
    l = random.choice([2, 3, 4, 5])  # 5 katmana kadar izin var
    population.append([f, l])

# Eğitim fonksiyonunu "Maraton"a göre güncelleyelim
def train_marathon(model, loader, epochs):
    model.to(device)
    optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
    criterion = nn.CrossEntropyLoss()

    model.train()
    for epoch in range(epochs):
        # Her epoch'ta veriyi dönüyoruz
        for data, target in loader:
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()

    # Eğitim bitince sınav yap (Test)
    model.eval()
    correct = 0; total = 0
    with torch.no_grad():
        for data, target in loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            _, predicted = torch.max(output.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()

    return 100 * correct / total

# --- BÜYÜK DÖNGÜ ---
start_time = time.time()

for generation in range(GENERATIONS):
    print(f"\n>>> NESİL {generation + 1} / {GENERATIONS}")
    scores = []

    for i, gene in enumerate(population):
        f, l = gene

        # Modeli yarat
        model = ProDynamicCNN(n_filters=f, n_layers=l)

        # Eğit (Maraton Modu)
        # Kullanıcı beklerken sıkılmasın diye print atalım
        print(f"  > Model {i+1} Eğitiliyor... (Mimari: {f} Filtre, {l} Katman)", end="")

        acc = train_marathon(model, client_a_loader, EPOCHS_PER_MODEL)

        print(f" -> Bitti. Başarım: %{acc:.2f}")
        scores.append((gene, acc))

    # Sıralama
    scores.sort(key=lambda x: x[1], reverse=True)
    best_gene = scores[0][0]
    best_acc = scores[0][1]

    print(f"  -> NESİL LİDERİ: {best_gene} (Acc: %{best_acc:.2f})")
    history.append(best_acc)

    # Mutasyon ve Yeni Nesil
    new_pop = [best_gene] # Lideri koru (Elitism)

    while len(new_pop) < POPULATION_SIZE:
        mutation_type = random.choice(['filter', 'layer'])
        new_f, new_l = best_gene

        if mutation_type == 'filter':
            # Filtreyi değiştir
            choices = [32, 64, 128]
            new_f = random.choice(choices)
        else:
            # Katmanı değiştir
            change = random.choice([-1, 1])
            new_l = new_l + change
            new_l = max(2, min(5, new_l)) # 2-5 arası sınır

        new_pop.append([new_f, new_l])

    population = new_pop

total_time = time.time() - start_time
print(f"\n--- SÜREÇ TAMAMLANDI ({total_time/60:.1f} Dakika) ---")
print(f"Bulunan EN GÜÇLÜ Mimari: {population[0]}")

--- MARATON BAŞLIYOR ---
Her model 5 Epoch eğitilecek.
Toplam 5 Nesil sürecek.
Bu işlem biraz zaman alacak, lütfen bekleyin...


>>> NESİL 1 / 5
  > Model 1 Eğitiliyor... (Mimari: 32 Filtre, 2 Katman) -> Bitti. Başarım: %36.00
  > Model 2 Eğitiliyor... (Mimari: 32 Filtre, 3 Katman) -> Bitti. Başarım: %31.10
  > Model 3 Eğitiliyor... (Mimari: 64 Filtre, 5 Katman) -> Bitti. Başarım: %47.90
  > Model 4 Eğitiliyor... (Mimari: 32 Filtre, 3 Katman) -> Bitti. Başarım: %26.70
  -> NESİL LİDERİ: [64, 5] (Acc: %47.90)

>>> NESİL 2 / 5
  > Model 1 Eğitiliyor... (Mimari: 64 Filtre, 5 Katman) -> Bitti. Başarım: %44.20
  > Model 2 Eğitiliyor... (Mimari: 64 Filtre, 5 Katman) -> Bitti. Başarım: %37.30
  > Model 3 Eğitiliyor... (Mimari: 32 Filtre, 5 Katman) -> Bitti. Başarım: %37.30
  > Model 4 Eğitiliyor... (Mimari: 32 Filtre, 5 Katman) -> Bitti. Başarım: %28.90
  -> NESİL LİDERİ: [64, 5] (Acc: %44.20)

>>> NESİL 3 / 5
  > Model 1 Eğitiliyor... (Mimari: 64 Filtre, 5 Katman) -> Bitti. Başarım: %40.50
 